### A Spam Classifier Attempt


In [1]:
import os

# Data prep
# train_dir = '/home/datadrive/PythonDev/datasets/Spam/raw/easy_ham'
# spam_train_dir = '/home/datadrive/PythonDev/datasets/Spam/raw/spam'
# processed_dir = '/home/datadrive/PythonDev/datasets/Spam/Processed/'
dataset_dir = '/home/datadrive/PythonDev/datasets/Spam/'
spam_zip = os.path.join(dataset_dir, 'mail_zip/spam')    
ham_zip = os.path.join(dataset_dir, 'mail_zip/ham')
ham_unzip = os.path.join(dataset_dir, 'mail_unziped/ham')
spam_unzip = os.path.join(dataset_dir, 'mail_unziped/spam')
        

In [2]:
import re
def html_cleaner(text):
    cleaned = re.sub(r"(?s)[\n]?", "", text)
    # Next remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", "", cleaned)
    # Finally deal with whitespace
    cleaned = re.sub(r" ", " ", cleaned)
    cleaned = re.sub(r"^$", " ", cleaned)
    cleaned = re.sub("''|,", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    return cleaned
    

**velocity** $= \begin{pmatrix}
10 \\
50 \\
5000 \\
\end{pmatrix}$


In [3]:
# function to analyse emails to determine content type, subject line


def read_subject(file, spam_ham):
    all_words = []
    subject = ''
    content_type = ''
    
    with open(file) as m:
        for i, line in enumerate(m):
            if subject == '':
                if line.rfind('Subject:',0) > -1:
                    subject = line[9:]
            if content_type == '':
                if line.rfind('Content-Type:', 0) > -1:
                    content_type = line[14:]
        return [file, spam_ham, subject, content_type]




In [4]:
import tarfile
def unzip_dir(path, extract_path):
    cnt = 0
    for zip in os.listdir(path):
        file_path = os.path.join(path, zip)
        print('Unziping ', file_path)
        cnt+=1
        tarobject = tarfile.open(file_path)
        tarobject.extractall(extract_path)
        tarobject.close()
        print('Extracted ', cnt, extract_path)
        
def extract_data():
    unziped_spams = os.listdir(os.path.join(dataset_dir, 'mail_unziped/spam'))
    
    if len(unziped_spams) == 0:        
        unzip_dir(spam_zip, spam_unzip)
        unzip_dir(ham_zip, ham_unzip)
    else:
        print(unziped_spams)
        print('Ziped files already extracted')
        pass       
    

In [5]:
def get_files_in_dir(path):
    fullpaths  = []
    for folder, subfolder, files in os.walk(path):
        for file in files:
            fullpaths.append(os.path.join(folder, file))
            # print(os.path.join(folder, file))
    return fullpaths

In [6]:
import numpy as np
from collections import Counter

def get_words_from_email(emailpaths):
    all_words = []
    cnt = 0;

    # cnt += 1
    # print(mail)
    for mailpath in emailpaths:
        raw_data = ''
        try:
            #print(mailpath[:1][0])
            raw_data = open(mailpath[:1][0], 'r', encoding='ascii', errors='surrogateescape').read()
            with open(mailpath[:1][0], 'r', encoding='ascii', errors='surrogateescape') as read:
                for line in read:
                    cleaned_line = html_cleaner(str(line))
                    #print(cleaned_line)
                    all_words += cleaned_line.split()
        except UnicodeDecodeError as ex:
            print(ex)
            print(raw_data)
 
    dictionary = Counter(all_words)    
    # remove single charactes
    lst_to_remove = [i for i in dictionary.keys()]
    cnt = len(lst_to_remove)
    print(cnt)
    # print(all_words)
    for i in range(cnt):
        item = lst_to_remove[i]
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    print('finished')
    words = [item.lower() for item in dictionary.keys()]
    return words

In [7]:
import pandas as pd
def split_train_test_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    train = [i for i in data.iloc[train_indices]]
    test = [i for i in data.iloc[test_indices]]
    return train, test
    

In [8]:
# Produce vectorized representation of each email.
def get_features(files, vectorizer):
    all_word_vec_shape = vectorizer.transform(['of the id and']).shape
    labels = []
    print(len(files))
    train_labels = np.zeros(len(files))
    vectorized_email = np.zeros((len(files), all_word_vec_shape[1]))
                            
    #vectorized_data  = np.zeros((len(files), all_word_vec_shape[1]))
    vectorized_email
    cnt = 0
    for email in files:
        # print(vectorizer.transform(email[:1]).toarray())
        vectorized_email += vectorizer.transform(email[:1]).toarray()
        #vectorized_data[cnt] = vectorizer.transform(email[:1]).toarray()
        labels += email[1:]
        # print(email[1:][0])
        train_labels[cnt] = email[1:][0] 
        vectorized_email
        cnt += 1
    return vectorized_email, labels, train_labels


### Start Processing

In [14]:
import pandas as pd

# Extract compressed files 
extract_data()

ham_paths = get_files_in_dir(ham_unzip)
ham_emails = [[f, 0] for f in ham_paths]
spam_emails = [[f, 1] for f in get_files_in_dir(spam_unzip)]

print(len(spam_emails), len(ham_emails))

# Combine spam and ham mails into one array
emails = spam_emails + ham_emails
len(emails)

# Set aside test and train sets
data = pd.Series(emails)
train_set_files, test_set_files = split_train_test_data(data, 0.2)


['spam_2', 'spam']
Ziped files already extracted
2464 7770


In [42]:
def get_body_from_mail(emailpaths):
    all_words = []
    cnt = 0;

    # cnt += 1
    # print(mail)
    mailpath = emailpaths
    print(mailpath)
    raw_data = ''
    past_header, lines = False, []
    try:
    #print(mailpath[:1][0])
    # raw_data = open(mailpath[:1][0], 'r', encoding='ascii', errors='surrogateescape').read()
    # print(raw_data)
    
        with open(mailpath[:1][0], 'r', encoding='ascii', errors='surrogateescape') as read:
            for line in read:
                if past_header:
                    lines.append(line)
                elif line == '\n':
                    past_header = True
                #print(cleaned_line)
    except UnicodeDecodeError as ex:
        print(ex)
        print(raw_data)
    return lines

#     dictionary = Counter(all_words)    
#     # remove single charactes
#     lst_to_remove = [i for i in dictionary.keys()]
#     cnt = len(lst_to_remove)
#     print(cnt)
#     # print(all_words)
#     for i in range(cnt):
#         item = lst_to_remove[i]
#         if item.isalpha() == False:
#             del dictionary[item]
#         elif len(item) == 1:
#             del dictionary[item]
#     print('finished')
#     words = [item.lower() for item in dictionary.keys()]
#    return words

In [43]:
datas = get_body_from_mail(train_set_files[2])

#words = get_words_from_email(train_set_files)

['/home/datadrive/PythonDev/datasets/Spam/mail_unziped/ham/easy_ham/00109.bcb73e4561798e05f2299471ab0be1bb', 0]


In [44]:
len(datas)

62

In [45]:
datas

['Seems fetchmail has a -a switch to get it all.\n',
 '\n',
 'Just need to install fetchmail now :)\n',
 '\n',
 'Glen\n',
 '\n',
 'On Mon, 2002-10-07 at 10:17, Stephane Dudzinski wrote:\n',
 '> Funny enough, that also happened to a Friend of mine who uses both the\n',
 '> web interface and a pop client. Last time i tried to send a picture\n',
 '> which was around 100k, it got denied saying that quota was exceeded.\n',
 '> When he looked at his account on the web, it mentionned 5 MB free, so i\n',
 "> have no idea what they're playing at ...\n",
 '> \n',
 "> Doesn't really help but just wanted to confirm the problem. \n",
 '> \n',
 '> Steph\n',
 '> \n',
 '> On Mon, 2002-10-07 at 10:10, Glen Gray wrote:\n',
 '> > Is there a way to get my read email downloaded off webmail.eircom.net.\n',
 '> > \n',
 "> > I've been reading the emails using the web based interface. But I've\n",
 "> > reached my quota limit. There doesn't seem to be any way to get the\n",
 '> > emails off the server. I can c

### Extract words for feature creation

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', tokenizer=None, preprocessor=None, stop_words=None)

# Get all words from emails
words = get_words_from_email(train_set_files)

reduced_words =[]
for word in words:
    if len(word) > 3 and len(word) <= 18:
        reduced_words.append(word)



219185
finished


In [13]:
        
fitted_words = vectorizer.fit_transform(reduced_words)



In [14]:
fitted_words

<41479x32153 sparse matrix of type '<class 'numpy.float64'>'
	with 41479 stored elements in Compressed Sparse Row format>

In [19]:
vectorizer.get_feature_names()


['aachen',
 'aalib',
 'aaliyah',
 'aall',
 'aanfh',
 'aaron',
 'aaronsw',
 'aarp',
 'aavid',
 'aavishkar',
 'aaxine',
 'abacha',
 'abandon',
 'abandoned',
 'abandoning',
 'abastecimientos',
 'abatement',
 'abbey',
 'abbreviated',
 'abbreviation',
 'abdominal',
 'abducted',
 'abduction',
 'abductions',
 'abdul',
 'abdullah',
 'abel',
 'abelard',
 'aber',
 'aberdeenshire',
 'aberration',
 'abetted',
 'abetting',
 'abide',
 'abidjan',
 'abilities',
 'ability',
 'abilitys',
 'abiltiy',
 'abilty',
 'abit',
 'abiword',
 'abject',
 'able',
 'ablility',
 'ably',
 'abnegate',
 'abnegateall',
 'abner',
 'abnormal',
 'abnormality',
 'abnormally',
 'aboard',
 'aboli',
 'abolins',
 'abolish',
 'abolished',
 'abolition',
 'abolute',
 'abonnement',
 'abonnements',
 'aboriginal',
 'aborigines',
 'abort',
 'aborted',
 'abortive',
 'abounds',
 'about',
 'above',
 'abovementioned',
 'abovetelefax',
 'abraham',
 'abreast',
 'abreau',
 'abridging',
 'abroad',
 'abruptly',
 'absconded',
 'absence',
 'absens

In [18]:
xy = vectorizer.transform(['aalib aachen allows alloy']).toarray()
xx = vectorizer.transform(['of the id and'])
xy

array([[ 0.50461003,  0.50461003,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
all_word_vec_shape = vectorizer.transform(['of the id and'])

all_word_vec_shape

In [ ]:
X_train, X_test, x_labels = get_features(train_set_files, vectorizer)
y_train, y_test, y_labels = get_features(test_set_files, vectorizer)

In [ ]:
print(y_test)

In [ ]:
from sklearn.svm import SVC, NuSVC, LinearSVC
model = SVC()
model.fit(X_train, X_test)

In [ ]:
result = model.predict(y_train)

In [ ]:
print(result)

In [ ]:
 y_train.shape

In [ ]:
for i in range(len(y_labels)):
    if result[i] == 1:
        print(result[i], y_labels[i])


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, result)